In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

/Users/admin/Projects/vocabulary_learning/notebooks
/Users/admin/Projects/vocabulary_learning


<IPython.core.display.Javascript object>

In [20]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)

import datetime
import dill

<IPython.core.display.Javascript object>

# Overall

In [21]:
historical_data_path = "data/raw/20201009/historical_data.csv"
vocab_path = "data/raw/20201009/german_english.csv"
dataset_path = "data/raw/20201009/dataset.pkl"

<IPython.core.display.Javascript object>

In [23]:
from src.data.make_dataset import create_dataset
create_dataset(historical_data_path, vocab_path, dataset_path)

Saved


<IPython.core.display.Javascript object>

# Details

In [22]:
def create_dataset(historical_data_path, vocab_path, dataset_path):

    historical_data = get_historical_data(historical_data_path)
    historical_data = create_historical_features(historical_data)

    vocab = get_vocab(vocab_path)
    vocab = create_vocab_features(vocab)

    dataset = merge_feature_datasets(historical_data, vocab)

    vardict = get_vardict()
    dataset = transform_type(dataset, vardict)

    with open(dataset_path, "wb") as file:
        dill.dump(dataset, file)

    print("Saved")

<IPython.core.display.Javascript object>

In [3]:
# Historical dataset
from src.data.get_dataset import get_historical_data
from src.data.make_historical_features import create_historical_features

historical_data = get_historical_data("data/raw/historical_data__large.csv")
historical_data = create_historical_features(historical_data)
historical_data

,id_vocab,german_word,english_word,score_before,score_before_other_language,language_asked,result,guess,question_time,write_it_again,confused_word,is_it_another_word,datetime,id_session,day,id_historical_data,past_occurrences_same_language,past_successes_same_language,past_fails_same_language,past_occurrences_any_language,past_successes_any_language,past_fails_any_language,days_since_last_occurrence_same_language,days_since_last_occurrence_any_language,days_since_last_success_same_language,days_since_last_success_any_language,previous_score,previous_score_other_language,week_number,day_week,hour,nb_words_session,days_since_first_occur_same_language,previous_result,previous_correct_article,previous_levenshtein_distance_guess_answer,previous_only_missed_uppercase,previous_question_time,previous_write_it_again_not_null,previous_write_it_again_german,previous_write_it_again_english,previous_confused_with_another_word,previous_confused_with_an_unknown_word,days_since_first_occur_any_language,previous_language_asked
0,61.0,währen,to last,0.0,0.0,german,0.0,enden,3.0,to last währen währen,NaN,0.0,2020-10-05 22:40:48.990953,0,2020-10-05,0,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,149.0,die Ausbildung,the training,0.0,0.0,english,0.0,the internship,6.0,die Ausbildung the training,NaN,0.0,2020-10-05 22:40:58.318949,0,2020-10-05,1,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,46.0,die Garantie,the guarantee,0.0,0.0,german,0.0,die Guarantee,3.0,the guarantee de Garantie die Grantie,NaN,0.0,2020-10-05 22:41:12.238464,0,2020-10-05,2,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,85.0,der Hahn,the rooster,0.0,0.0,english,0.0,the chicken,9.0,der Hahn the rooster der Hahn,NaN,0.0,2020-10-05 22:41:28.765347,0,2020-10-05,3,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,71.0,beachten,to pay attention,0.0,0.0,german,0.0,,2.0,beachten to pay attention beachten,NaN,0.0,2020-10-05 22:41:44.138936,0,2020-10-05,4,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,5,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,9.0,Deutsch,German,0.0,0.0,german,1.0,Deutsch,3.0,NaN,NaN,NaN,2020-10-08 17:11:12.760509,5,2020-10-08,315,0,0.0,0.0,2,1.0,1.0,NaT,1 days,NaT,1 days,0.0,0.0,41,4,17,96,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 days,english
316,18.0,reservieren,to reserve,0.0,0.0,english,1.0,to reserve,3.0,NaN,NaN,NaN,2020-10-08 17:11:15.848705,5,2020-10-08,316,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,4,17,97,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
317,52.0,der Cousin,the cousin,1.0,1.0,english,1.0,the cousin,4.0,NaN,NaN,NaN,2020-10-08 17:11:19.535079,5,2020-10-08,317,1,1.0,0.0,2,2.0,0.0,1 days,1 days,1 days,1 days,1.0,1.0,41,4,17,98,0 days,1.0,None,0,None,3.0,False,NaN,NaN,False,NaN,1 days,english
318,113.0,der Magen,the stomach,0.0,0.0,german,1.0,der Magen,3.0,NaN,NaN,NaN,2020-10-08 17:11:23.691143,5,2020-10-08,318,2,1.0,1.0,2,1.0,1.0,1 days,1 days,1 days,1 days,0.0,0.0,41,4,17,99,1 days,1.0,True,0,False,3.0,False,NaN,NaN,False,NaN,1 days,german


<IPython.core.display.Javascript object>

In [4]:
# Vocab dataset
from src.data.get_dataset import get_vocab
from src.data.make_vocab_features import create_vocab_features

vocab = get_vocab("data/raw/german_english.csv")
vocab = create_vocab_features(vocab)
vocab

,id_vocab,levenshtein_distance_german_english,nb_characters_german,nb_characters_english,nb_words_german,nb_words_english,is_noun,is_verb,difficulty_category
0,1,7,8,4,1,1,False,False,-4
1,1,7,8,4,1,1,False,False,5
2,2,0,9,9,1,1,True,False,5
3,3,4,4,5,1,1,False,False,5
4,4,6,10,9,1,1,True,False,4
...,...,...,...,...,...,...,...,...,...
172,156,4,6,6,1,1,False,False,-10
173,157,8,13,9,1,1,True,False,-10
174,158,7,12,9,1,1,True,False,-10
175,159,3,10,11,1,1,True,False,-10


<IPython.core.display.Javascript object>

In [5]:
# Dataset
from src.data.make_dataset import merge_feature_datasets

dataset = merge_feature_datasets(historical_data, vocab)
dataset

,id_vocab,german_word,english_word,score_before,score_before_other_language,language_asked,result,guess,question_time,write_it_again,confused_word,is_it_another_word,datetime,id_session,day,id_historical_data,past_occurrences_same_language,past_successes_same_language,past_fails_same_language,past_occurrences_any_language,past_successes_any_language,past_fails_any_language,days_since_last_occurrence_same_language,days_since_last_occurrence_any_language,days_since_last_success_same_language,days_since_last_success_any_language,previous_score,previous_score_other_language,week_number,day_week,hour,nb_words_session,days_since_first_occur_same_language,previous_result,previous_correct_article,previous_levenshtein_distance_guess_answer,previous_only_missed_uppercase,previous_question_time,previous_write_it_again_not_null,previous_write_it_again_german,previous_write_it_again_english,previous_confused_with_another_word,previous_confused_with_an_unknown_word,days_since_first_occur_any_language,previous_language_asked,levenshtein_distance_german_english,nb_characters_german,nb_characters_english,nb_words_german,nb_words_english,is_noun,is_verb,difficulty_category
0,61.0,währen,to last,0.0,0.0,german,0.0,enden,3.0,to last währen währen,NaN,0.0,2020-10-05 22:40:48.990953,0,2020-10-05,0,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,6,6,7,1,1,False,True,-1
3,149.0,die Ausbildung,the training,0.0,0.0,english,0.0,the internship,6.0,die Ausbildung the training,NaN,0.0,2020-10-05 22:40:58.318949,0,2020-10-05,1,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,7,14,12,1,1,True,False,-9
7,46.0,die Garantie,the guarantee,0.0,0.0,german,0.0,die Guarantee,3.0,the guarantee de Garantie die Grantie,NaN,0.0,2020-10-05 22:41:12.238464,0,2020-10-05,2,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2,12,13,1,1,True,False,1
8,85.0,der Hahn,the rooster,0.0,0.0,english,0.0,the chicken,9.0,der Hahn the rooster der Hahn,NaN,0.0,2020-10-05 22:41:28.765347,0,2020-10-05,3,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,7,8,11,1,1,True,False,-3
10,71.0,beachten,to pay attention,0.0,0.0,german,0.0,,2.0,beachten to pay attention beachten,NaN,0.0,2020-10-05 22:41:44.138936,0,2020-10-05,4,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,1,22,5,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,10,8,16,1,2,False,True,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,18.0,reservieren,to reserve,0.0,0.0,english,1.0,to reserve,3.0,NaN,NaN,NaN,2020-10-08 17:11:15.848705,5,2020-10-08,316,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,4,17,97,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,4,11,10,1,1,False,True,4
99,52.0,der Cousin,the cousin,1.0,1.0,english,1.0,the cousin,4.0,NaN,NaN,NaN,2020-10-08 17:11:19.535079,5,2020-10-08,317,1,1.0,0.0,2,2.0,0.0,1 days,1 days,1 days,1 days,1.0,1.0,41,4,17,98,0 days,1.0,None,0,None,3.0,False,NaN,NaN,False,NaN,1 days,english,0,10,10,1,1,True,False,1
98,52.0,der Cousin,the cousin,1.0,1.0,english,1.0,the cousin,4.0,NaN,NaN,NaN,2020-10-08 17:11:19.535079,5,2020-10-08,317,1,1.0,0.0,2,2.0,0.0,1 days,1 days,1 days,1 days,1.0,1.0,41,4,17,98,0 days,1.0,None,0,None,3.0,False,NaN,NaN,False,NaN,1 days,english,0,10,10,1,1,True,False,0
245,113.0,der Magen,the stomach,0.0,0.0,german,1.0,der Magen,3.0,NaN,NaN,NaN,2020-10-08 17:11:23.691143,5,2020-10-08,318,2,1.0,1.0,2,1.0,1.0,1 days,1 days,1 days,1 days,0.0,0.0,41,4,17,99,1 days,1.0,True,0,False,3.0,False,NaN,NaN,False,NaN,1 days,german,6,9,11,1,1,True,False,-6


<IPython.core.display.Javascript object>

In [6]:
# Vardict
from src.data.make_dataset import get_vardict

vardict = get_vardict()
vardict

{'target': 'result',
 'numerical': ['nb_characters_german',
  'nb_characters_english',
  'nb_words_german',
  'nb_words_english',
  'levenshtein_distance_german_english',
  'previous_score',
  'previous_score_other_language',
  'previous_levenshtein_distance_guess_answer',
  'previous_question_time',
  'previous_write_it_again_german',
  'previous_write_it_again_english',
  'past_occurrences_same_language',
  'past_successes_same_language',
  'past_fails_same_language',
  'past_occurrences_any_language',
  'past_successes_any_language',
  'past_fails_any_language',
  'week_number',
  'day_week',
  'hour',
  'nb_words_session',
  'difficulty_category'],
 'diff_time': ['days_since_last_occurrence_same_language',
  'days_since_last_occurrence_any_language',
  'days_since_last_success_same_language',
  'days_since_last_success_any_language',
  'days_since_first_occur_same_language',
  'days_since_first_occur_any_language'],
 'boolean': ['previous_result',
  'previous_correct_article',
  'p

<IPython.core.display.Javascript object>

In [7]:
# Transform datatype
from src.data.make_dataset import transform_type

dataset = transform_type(dataset, vardict)
dataset

,id_vocab,german_word,english_word,score_before,score_before_other_language,language_asked,result,guess,question_time,write_it_again,confused_word,is_it_another_word,datetime,id_session,day,id_historical_data,past_occurrences_same_language,past_successes_same_language,past_fails_same_language,past_occurrences_any_language,past_successes_any_language,past_fails_any_language,days_since_last_occurrence_same_language,days_since_last_occurrence_any_language,days_since_last_success_same_language,days_since_last_success_any_language,previous_score,previous_score_other_language,week_number,day_week,hour,nb_words_session,days_since_first_occur_same_language,previous_result,previous_correct_article,previous_levenshtein_distance_guess_answer,previous_only_missed_uppercase,previous_question_time,previous_write_it_again_not_null,previous_write_it_again_german,previous_write_it_again_english,previous_confused_with_another_word,previous_confused_with_an_unknown_word,days_since_first_occur_any_language,previous_language_asked,levenshtein_distance_german_english,nb_characters_german,nb_characters_english,nb_words_german,nb_words_english,is_noun,is_verb,difficulty_category
0,61.0,währen,to last,0.0,0.0,german,0.0,enden,3.0,to last währen währen,NaN,0.0,2020-10-05 22:40:48.990953,0,2020-10-05,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,41.0,1.0,22.0,1.0,NaN,None,None,NaN,None,NaN,None,NaN,NaN,None,None,NaN,None,6.0,6.0,7.0,1.0,1.0,0.0,1.0,-1.0
3,149.0,die Ausbildung,the training,0.0,0.0,english,0.0,the internship,6.0,die Ausbildung the training,NaN,0.0,2020-10-05 22:40:58.318949,0,2020-10-05,1,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,41.0,1.0,22.0,2.0,NaN,None,None,NaN,None,NaN,None,NaN,NaN,None,None,NaN,None,7.0,14.0,12.0,1.0,1.0,1.0,0.0,-9.0
7,46.0,die Garantie,the guarantee,0.0,0.0,german,0.0,die Guarantee,3.0,the guarantee de Garantie die Grantie,NaN,0.0,2020-10-05 22:41:12.238464,0,2020-10-05,2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,41.0,1.0,22.0,3.0,NaN,None,None,NaN,None,NaN,None,NaN,NaN,None,None,NaN,None,2.0,12.0,13.0,1.0,1.0,1.0,0.0,1.0
8,85.0,der Hahn,the rooster,0.0,0.0,english,0.0,the chicken,9.0,der Hahn the rooster der Hahn,NaN,0.0,2020-10-05 22:41:28.765347,0,2020-10-05,3,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,41.0,1.0,22.0,4.0,NaN,None,None,NaN,None,NaN,None,NaN,NaN,None,None,NaN,None,7.0,8.0,11.0,1.0,1.0,1.0,0.0,-3.0
10,71.0,beachten,to pay attention,0.0,0.0,german,0.0,,2.0,beachten to pay attention beachten,NaN,0.0,2020-10-05 22:41:44.138936,0,2020-10-05,4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,41.0,1.0,22.0,5.0,NaN,None,None,NaN,None,NaN,None,NaN,NaN,None,None,NaN,None,10.0,8.0,16.0,1.0,2.0,0.0,1.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,18.0,reservieren,to reserve,0.0,0.0,english,1.0,to reserve,3.0,NaN,NaN,NaN,2020-10-08 17:11:15.848705,5,2020-10-08,316,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,41.0,4.0,17.0,97.0,NaN,None,None,NaN,None,NaN,None,NaN,NaN,None,None,NaN,None,4.0,11.0,10.0,1.0,1.0,0.0,1.0,4.0
99,52.0,der Cousin,the cousin,1.0,1.0,english,1.0,the cousin,4.0,NaN,NaN,NaN,2020-10-08 17:11:19.535079,5,2020-10-08,317,1.0,1.0,0.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,41.0,4.0,17.0,98.0,0.0,True,None,0.0,None,3.0,False,NaN,NaN,False,None,1.0,english,0.0,10.0,10.0,1.0,1.0,1.0,0.0,1.0
98,52.0,der Cousin,the cousin,1.0,1.0,english,1.0,the cousin,4.0,NaN,NaN,NaN,2020-10-08 17:11:19.535079,5,2020-10-08,317,1.0,1.0,0.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,41.0,4.0,17.0,98.0,0.0,True,None,0.0,None,3.0,False,NaN,NaN,False,None,1.0,english,0.0,10.0,10.0,1.0,1.0,1.0,0.0,0.0
245,113.0,der Magen,the stomach,0.0,0.0,german,1.0,der Magen,3.0,NaN,NaN,NaN,2020-10-08 17:11:23.691143,5,2020-10-08,318,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,41.0,4.0,17.0,99.0,1.0,True,True,0.0,False,3.0,False,NaN,NaN,False,N

<IPython.core.display.Javascript object>

In [8]:
[x for x in dataset.columns.tolist() if x not in vardict["all"]]

['id_vocab',
 'german_word',
 'english_word',
 'score_before',
 'score_before_other_language',
 'result',
 'guess',
 'question_time',
 'write_it_again',
 'confused_word',
 'is_it_another_word',
 'datetime',
 'id_session',
 'day',
 'id_historical_data']

<IPython.core.display.Javascript object>

In [11]:
dataset[dataset["german_word"] == "oft"][
    ["german_word", "english_word"] + [vardict["target"]] + vardict["all"]
]

,german_word,english_word,result,nb_characters_german,nb_characters_english,nb_words_german,nb_words_english,levenshtein_distance_german_english,previous_score,previous_score_other_language,previous_levenshtein_distance_guess_answer,previous_question_time,previous_write_it_again_german,previous_write_it_again_english,past_occurrences_same_language,past_successes_same_language,past_fails_same_language,past_occurrences_any_language,past_successes_any_language,past_fails_any_language,week_number,day_week,hour,nb_words_session,difficulty_category,days_since_last_occurrence_same_language,days_since_last_occurrence_any_language,days_since_last_success_same_language,days_since_last_success_any_language,days_since_first_occur_same_language,days_since_first_occur_any_language,previous_result,previous_correct_article,previous_only_missed_uppercase,previous_write_it_again_not_null,is_noun,is_verb,previous_confused_with_another_word,previous_confused_with_an_unknown_word,language_asked,previous_language_asked
227,oft,often,0.0,3.0,5.0,1.0,1.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,41.0,2.0,13.0,25.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0.0,0.0,None,None,german,None
228,oft,often,1.0,3.0,5.0,1.0,1.0,2.0,0.0,-1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1.0,41.0,3.0,10.0,18.0,3.0,NaN,1.0,NaN,NaN,NaN,0.0,None,None,None,None,0.0,0.0,None,None,english,german
229,oft,often,1.0,3.0,5.0,1.0,1.0,2.0,1.0,-1.0,0.0,2.0,NaN,NaN,1.0,1.0,0.0,2.0,1.0,1.0,41.0,4.0,16.0,14.0,3.0,1.0,1.0,1.0,1.0,0.0,1.0,True,None,None,False,0.0,0.0,False,None,english,english


<IPython.core.display.Javascript object>

In [12]:
historical_data[historical_data["german_word"] == "oft"]

,id_vocab,german_word,english_word,score_before,score_before_other_language,language_asked,result,guess,question_time,write_it_again,confused_word,is_it_another_word,datetime,id_session,day,id_historical_data,past_occurrences_same_language,past_successes_same_language,past_fails_same_language,past_occurrences_any_language,past_successes_any_language,past_fails_any_language,days_since_last_occurrence_same_language,days_since_last_occurrence_any_language,days_since_last_success_same_language,days_since_last_success_any_language,previous_score,previous_score_other_language,week_number,day_week,hour,nb_words_session,days_since_first_occur_same_language,previous_result,previous_correct_article,previous_levenshtein_distance_guess_answer,previous_only_missed_uppercase,previous_question_time,previous_write_it_again_not_null,previous_write_it_again_german,previous_write_it_again_english,previous_confused_with_another_word,previous_confused_with_an_unknown_word,days_since_first_occur_any_language,previous_language_asked
93,23.0,oft,often,0.0,0.0,german,0.0,seltsam,7.0,often oft oft oft,25.0,True,2020-10-06 13:39:01.702693,2,2020-10-06,93,0,0.0,0.0,0,0.0,0.0,NaT,NaT,NaT,NaT,0.0,0.0,41,2,13,25,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
137,23.0,oft,often,0.0,-1.0,english,1.0,often,2.0,NaN,NaN,NaN,2020-10-07 10:17:50.097847,3,2020-10-07,137,0,0.0,0.0,1,0.0,1.0,NaT,1 days,NaT,NaT,0.0,-1.0,41,3,10,18,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days,german
233,23.0,oft,often,1.0,-1.0,english,1.0,often,2.0,NaN,NaN,NaN,2020-10-08 16:56:18.644200,5,2020-10-08,233,1,1.0,0.0,2,1.0,1.0,1 days,1 days,1 days,1 days,1.0,-1.0,41,4,16,14,0 days,1.0,None,0,None,2.0,False,NaN,NaN,False,NaN,1 days,english


<IPython.core.display.Javascript object>

In [ ]:
# use dill - works the same way as pickle

with open('data/interim/{}__Q.pkl'.format('MC_every_visit'), 'wb') as file:
    dill.dump(Q, file)

# Historical dataset

In [ ]:
from src.data.get_dataset import get_historical_data

historical_data_test = get_historical_data("data/raw/historical_data__feature.csv")
historical_data_test

## Add features

In [ ]:
from src.data.make_historical_features import create_historical_features

historical_data_test = create_historical_features(historical_data_test)
historical_data_test

# Vocab dataset

In [ ]:
from src.data.get_dataset import get_vocab

vocab = get_vocab("data/raw/german_english__feature.csv")
vocab

## Add features

In [ ]:
from src.data.make_vocab_features import create_vocab_features

vocab_test = create_vocab_features(vocab)
vocab_test

# Mix datasets

In [ ]:
dataset = pd.merge(historical_data_test, vocab_test, on="id_vocab")
dataset.sort_values("datetime", inplace=True)
dataset

# Variable type transformation

In [ ]:
# List of feature columns

In [ ]:
dataset.columns.tolist()

In [ ]:
vardict = dict()

## Target

In [ ]:
vardict["target"] = "result"

In [ ]:
dataset[[vardict["target"]]]

## Numerical

In [ ]:
vardict["numerical"] = [
    "previous_occurrences_same_language",
    "previous_successes_same_language",
    "previous_fails_same_language",
    "previous_occurrences_any_language",
    "previous_successes_any_language",
    "previous_fails_any_language",
    "levenshtein_distance_guess_answer",
    "previous_question_time",
    "write_it_again_german",
    "write_it_again_english",
    "levenshtein_distance_german_english",
]

In [ ]:
for i_num_var in vardict["numerical"]:
    dataset[i_num_var] = dataset[i_num_var].astype(float)

In [ ]:
dataset[["german_word", "guess"] + vardict["numerical"]]

## Difference in time

In [ ]:
vardict["diff_time"] = [
    "days_since_last_occurrence_same_language",
    "days_since_last_occurrence_any_language",
    "days_since_last_success_same_language",
    "days_since_last_success_any_language",
    "days_since_first_occur_same_language",
    "days_since_first_occur_any_language",
]

In [ ]:
for i_diff_time_var in vardict["diff_time"]:
    dataset[i_diff_time_var] = dataset[i_diff_time_var].dt.days.astype(float)

In [ ]:
dataset[["german_word", "guess"] + vardict["diff_time"]]

## Boolean

In [ ]:
vardict["boolean"] = [
    "previous_result",
    "correct_article",
    "only_missed_uppercase",
    "write_it_again_not_null",
]

In [ ]:
for i_boolean_var in vardict["boolean"]:
    dataset.loc[~dataset[i_boolean_var].isna(), i_boolean_var] = dataset.loc[
        ~dataset[i_boolean_var].isna(), i_boolean_var
    ].astype("bool")

In [ ]:
dataset[["german_word", "guess"] + vardict["boolean"]]

## Categorical

In [ ]:
vardict["categorical"] = [
    "previous_language_asked",
]

In [ ]:
for i_categorical_var in vardict["categorical"]:
    dataset.loc[~dataset[i_categorical_var].isna(), i_categorical_var] = dataset.loc[
        ~dataset[i_categorical_var].isna(), i_categorical_var
    ].astype(str)

    dataset.loc[dataset[i_categorical_var].isna(), i_categorical_var] = None

In [ ]:
dataset[["german_word", "guess"] + vardict["categorical"]]